In [1]:
#### Basic Setting

import  load_dotenv
load_dotenv.load_dotenv("../../All_LLM_tutorial/.env")

True

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.core.node_parser import SentenceSplitter


Settings.llm = OpenAI(model="gpt-4o-mini")
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads https://huggingface.co/BAAI/bge-small-en-v1.5
# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
Settings.embed_model = OpenAIEmbedding()

/home/work/enssel_test/yhkim/llamaindex_study/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
chroma_client = chromadb.PersistentClient(path="./chroma_db_example")
chroma_collection = chroma_client.get_or_create_collection("cloud")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)


In [4]:
from IPython.display import Markdown, display


# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

# Routing

In [5]:
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

In [8]:
# pydantic selectors feed in pydantic objects to a function calling API
# single selector (pydantic, function calling)
# selector = PydanticSingleSelector.from_defaults()

# multi selector (pydantic, function calling)
# selector = PydanticMultiSelector.from_defaults()

# LLM selectors use text completion endpoints
# single selector (LLM)
# selector = LLMSingleSelector.from_defaults()
# multi selector (LLM)
selector = LLMMultiSelector.from_defaults()

In [6]:
from llama_index.core.tools import ToolMetadata

tool_choices = [
    ToolMetadata(
        name="covid_nyt",
        description=("This tool contains a NYT news article about COVID-19"),
    ),
    ToolMetadata(
        name="covid_wiki",
        description=("This tool contains the Wikipedia page about COVID-19"),
    ),
    ToolMetadata(
        name="covid_tesla",
        description=("This tool contains the Wikipedia page about apples"),
    ),
]

In [9]:
display_prompt_dict(selector.get_prompts())

**Prompt Key**: prompt<br>**Text:** <br>

Some choices are given below. It is provided in a numbered list (1 to {num_choices}), where each item in the list corresponds to a summary.
---------------------
{context_list}
---------------------
Using only the choices above and not prior knowledge, return the top choices (no more than {max_outputs}, but only select what is needed) that are most relevant to the question: '{query_str}'


The output should be ONLY JSON formatted as a JSON instance.

Here is an example:
[
    {{
        choice: 1,
        reason: "<insert reason for choice>"
    }},
    ...
]



<br><br>

In [11]:
selector_result = selector.select(
    tool_choices, query="Tell me more about COVID-19"
)

In [12]:
selector_result.selections

[SingleSelection(index=0, reason='This choice contains a news article about COVID-19, providing current information and updates.'),
 SingleSelection(index=1, reason='This choice contains the Wikipedia page about COVID-19, which offers comprehensive information and background on the topic.')]

# Query Rewriting

In [13]:
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI

query_gen_str = """\
You are a helpful assistant that generates multiple search queries based on a \
single input query. Generate {num_queries} search queries, one on each line, \
related to the following input query:
Query: {query}
Queries:
"""
query_gen_prompt = PromptTemplate(query_gen_str)

llm = OpenAI(model="gpt-4o-mini")


def generate_queries(query: str, llm, num_queries: int = 4):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    # assume LLM proper put each query on a newline
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries

In [14]:
queries = generate_queries("지금 당장 퇴근하는 방법은?", llm)

Generated queries:
1. 퇴근하는 방법과 팁
2. 지금 퇴근할 수 있는 방법
3. 퇴근 시간 단축하는 방법
4. 퇴근 후 할 일 추천


## Query Rewriting (using QueryTransform)

In [15]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.llms.openai import OpenAI

In [16]:
hyde = HyDEQueryTransform(include_original=True)
llm = OpenAI(model="gpt-4o-mini")

query_bundle = hyde.run("RAG 기술 설명해줘")

In [20]:
query_bundle.custom_embedding_strs

['RAG( Retrieval-Augmented Generation) 기술은 자연어 처리(NLP) 분야에서 정보 검색과 생성 모델을 결합한 혁신적인 접근 방식입니다. 이 기술은 주로 대규모 언어 모델(예: GPT-3, BERT 등)과 정보 검색 시스템을 통합하여, 사용자가 질문을 했을 때 보다 정확하고 관련성 높은 답변을 제공하는 데 초점을 맞추고 있습니다.\n\nRAG의 작동 방식은 크게 두 단계로 나눌 수 있습니다. 첫 번째 단계는 정보 검색 단계로, 사용자의 질문에 대한 관련 문서나 정보를 데이터베이스에서 검색하는 것입니다. 이 과정에서는 일반적으로 TF-IDF, BM25와 같은 전통적인 정보 검색 기법이나, 최근에는 딥러닝 기반의 검색 모델이 사용됩니다. 검색된 문서들은 사용자의 질문과 관련된 내용을 포함하고 있어야 합니다.\n\n두 번째 단계는 생성 단계로, 검색된 정보를 바탕으로 자연어 생성 모델이 최종 답변을 생성하는 것입니다. 이 단계에서는 검색된 문서의 내용을 요약하거나, 질문에 대한 구체적인 답변을 생성하는 데 사용됩니다. RAG 모델은 이러한 두 단계를 통합하여, 정보의 정확성과 생성의 유창성을 동시에 확보할 수 있습니다.\n\nRAG 기술의 장점은 정보의 신뢰성을 높이고, 사용자가 원하는 정보를 보다 정확하게 제공할 수 있다는 점입니다. 또한, 대규모 데이터셋에서 학습된 언어 모델이기 때문에, 다양한 주제에 대해 유연하게 대응할 수 있는 능력을 가지고 있습니다. 이러한 특성 덕분에 RAG는 고객 지원, 질의응답 시스템, 콘텐츠 생성 등 다양한 분야에서 활용되고 있습니다. \n\n결론적으로, RAG 기술은 정보 검색과 자연어 생성을 결합하여, 보다 효과적이고 정확한 정보 제공을 가능하게 하는 혁신적인 방법론으로 자리잡고 있습니다.',
 'RAG 기술 설명해줘']

In [21]:
from llama_index.core.question_gen import LLMQuestionGenerator
from llama_index.question_gen.openai import OpenAIQuestionGenerator
from llama_index.llms.openai import OpenAI

In [22]:
llm = OpenAI()
question_gen = OpenAIQuestionGenerator.from_defaults(llm=llm)

In [23]:
display_prompt_dict(question_gen.get_prompts())

**Prompt Key**: question_gen_prompt<br>**Text:** <br>

You are a world class state of the art agent.

You have access to multiple tools, each representing a different data source or API.
Each of the tools has a name and a description, formatted as a JSON dictionary.
The keys of the dictionary are the names of the tools and the values are the descriptions.
Your purpose is to help answer a complex user question by generating a list of sub questions that can be answered by the tools.

These are the guidelines you consider when completing your task:
* Be as specific as possible
* The sub questions should be relevant to the user question
* The sub questions should be answerable by the tools provided
* You can generate multiple sub questions for each tool
* Tools must be specified by their name, not their description
* You don't need to use a tool if you don't think it's relevant

Output the list of sub questions by calling the SubQuestionList function.

## Tools
```json
{tools_str}
```

## User Question
{query_str}



<br><br>

In [24]:
from llama_index.core.tools import ToolMetadata

tool_choices = [
    ToolMetadata(
        name="uber_2021_10k",
        description=(
            "Provides information about Uber financials for year 2021"
        ),
    ),
    ToolMetadata(
        name="lyft_2021_10k",
        description=(
            "Provides information about Lyft financials for year 2021"
        ),
    ),
]

In [25]:
from llama_index.core import QueryBundle

query_str = "Compare and contrast Uber and Lyft"
choices = question_gen.generate(tool_choices, QueryBundle(query_str=query_str))

In [26]:
choices

[SubQuestion(sub_question='What were the total revenues for Uber in 2021?', tool_name='uber_2021_10k'),
 SubQuestion(sub_question='What were the total revenues for Lyft in 2021?', tool_name='lyft_2021_10k'),
 SubQuestion(sub_question='What were the net profits for Uber in 2021?', tool_name='uber_2021_10k'),
 SubQuestion(sub_question='What were the net profits for Lyft in 2021?', tool_name='lyft_2021_10k')]

In [27]:
from llama_index.core.agent import ReActChatFormatter
from llama_index.core.agent.react.output_parser import ReActOutputParser
from llama_index.core.tools import FunctionTool
from llama_index.core.llms import ChatMessage

In [28]:
def execute_sql(sql: str) -> str:
    """Given a SQL input string, execute it."""
    # NOTE: This is a mock function
    return f"Executed {sql}"


def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


tool1 = FunctionTool.from_defaults(fn=execute_sql)
tool2 = FunctionTool.from_defaults(fn=add)
tools = [tool1, tool2]

In [29]:
chat_formatter = ReActChatFormatter()
output_parser = ReActOutputParser()
input_msgs = chat_formatter.format(
    tools,
    [
        ChatMessage(
            content="Can you find the top three rows from the table named `revenue_years`",
            role="user",
        )
    ],
)
input_msgs

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: execute_sql\nTool Description: execute_sql(sql: str) -> str\nGiven a SQL input string, execute it.\nTool Args: {"properties": {"sql": {"title": "Sql", "type": "string"}}, "required": ["sql"], "type": "object"}\n\n> Tool Name: add\nTool Description: add(a: int, b: int) -> int\nAdd two numbers.\nTool Args: {"properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "integer"}}, "required": ["a", "b"], "type":